<a href="https://colab.research.google.com/github/naoya5614/Kaggle/blob/main/Future_Prediction_Of_Cloud_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 雲画像の未来予測

# [1] データの確認・把握

## 1.衛星画像データの確認・把握

### 1-1.衛星画像データのファイルパス

**ファイルパスとは、ファイルの住所を表す文字列を意味し、以下の2つに大別される**
```
*   絶対パス: 最上位の階層から見た目的のファイルの場所
*   相対パス: プログラムを実行する位置から見た目的のファイルの場所
```

### 1-2.衛星画像を読み込む

**画像の読み込むライブラリのインポート**
```
# OpenCVライブラリのインポート
import cv2
```
**画像を読み込む**
```
# cv2.imread()関数を使って、白黒画像を読み込む
image = cv2.imread(画像のファイルパス, 0)
```

In [ ]:
# ライブラリのインポート
import numpy as np
import cv2

# 2016年１月１日16時時点の衛星画像を指定するファイルパス
file_path = 'train/sat/2016-01-01/2016-01-01-16-00.fv.png'

# 衛星画像を読み込む
image = cv2.imread(file_path, 0)

# 出力
print(image)

### 1-3.衛星画像を構成する値の統計量を確認する

**読み込んだ衛星画像がどのような値で構成されているのかを確認する**
```
*   最大値　np.max()関数
*   最小値　np.min()関数
*   平均値　np.mean()関数
```
```
# NumPyライブラリのインポート
import numpy as np

# 最大値の取得
np.max(配列)

# 最小値の取得
np.min(配列)

# 平均値の取得
np.mean(配列)
```

In [ ]:
# ライブラリのインポート
import numpy as np
import cv2

# 2016年１月１日16時時点の衛星画像を指定するファイルパス
file_path = 'train/sat/2016-01-01/2016-01-01-16-00.fv.png'

# 衛星画像を読み込む
image = cv2.imread(file_path, 0)

# 最大値の出力
print(np.max(image))

# 最小値の出力
print(np.min(image))

# 平均値の出力
print(np.mean(image))

### 1-4.衛星画像を可視化する

**今回扱う衛星画像**
```
*   最小値0
*   最大値255
# 8bit(256階調)
```
**可視化ライブラリをインポートする**
```
# matplotlib.pyplotモジュールのインポート
import matplotlib.pyplot as plt
```
**画像を可視化する**
```
# 白黒画像の描画
plt.imshow(画像を表す配列, 'gray')
```

In [ ]:
# ライブラリのインポート
import numpy as np
import cv2
import matplotlib.pyplot as plt

# 2016年１月１日16時時点の衛星画像を指定するファイルパス
file_path = 'train/sat/2016-01-01/2016-01-01-16-00.fv.png'

# 衛星画像を読み込む
image = cv2.imread(file_path, 0)

# 衛星画像を描画する
plt.imshow(image, 'gray')

# グラフの出力
plt.show()

### 1-5.ファイルパスを一般化する① f文字列による文字列の置換

**規則性を捉えるために、文字列の構成成分を理解する**
```
# 下記のように一般化する
{trainもしくはtest}/sat/{年}-{月}-{日}/{年}-{月}-{日}-{時}-00.fv.png

衛星画像のファイルパスは
*   train もしくは test
*   年
*   月
*   日
*   時
の5つの成分から構成されています。


これらの成分に対応する５つの変数である
*   phase:train もしくはtest
*   year: 年
*   month: 月
*   day: 日
*   hour: 時間
を与える。

{phase}/sat/{year}-{month}-{day}/{year}-{month}-{day}-{hour}-00.fv.png
```

**f文字列と置換フィールド**
```
phase = 'train'
year = 2016
month = 1
day = 1
hour = 16

# f文字列による文字列の置換
file_path = f'{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png'

## 置換フィールド{}内の文字列が変数によって置換されることで、
## file_pathに代入される文字列は、'train/sat/2016-01-01/2016-01-01-16-00.fv.png'となる。
```

In [ ]:
# 衛星画像のファイルパスを特定するための５つの変数に値を代入する
phase = 'train'
year = 2016
month = 1
day = 1
hour = 16

# 「f文字列」による置換が可能な文字列をfile_pathに代入する
file_path = f'{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png'

print(file_path)

### 1-6.ファイルパスを一般化する② 日時の扱い方

**衛星画像の各ファイルパスを分ける差分となる変数について改めて確認する**
```
*   phase:train もしくはtest
*   year: 年
*   month: 月
*   day: 日
*   hour: 時間
# year、month、day、hourの４つは全て日時を表すもの
# phaseに関しても日時の情報によって文字列を特定できる
```
**datetimeとtimedelta**
```
datetime: 日時情報を表す
timedelta: 日時どうしの差分(経過時間)を表す
```
**datetime、timedeltaのインポート**
```
# datetime.datetimeクラスのインポート
from datetime import datetime as dt

# datetime.timedeltaクラスのインポート
from datetime import timedelta
```

**1. datetimeによる日時の操作**

```
# オブジェクト生成の際の引数に(年, 月, 日, 時, 分, 秒)をそれぞれ指定することで、日時を表すオブジェクトを作成できる

# 「2016年1月1日16時0分0秒」
date = dt(2016, 1, 1, 16, 0, 0)
```



**年, 月, 日, 時, 分, 秒の情報を、それぞれ単独で数値データとして抽出する**

```
# 「2016年1月1日16時0分0秒」
date = dt(2016, 1, 1, 16, 0, 0)


# dateから、「年」のみを数値データとして抽出する
## yearに2016が代入される
year = date.year

# dateから、「月」のみを数値データとして抽出する
## monthに1が代入される
month = date.month

# dateから、「日」のみを数値データとして抽出する
## dayに1が代入される
day = date.day

# dateから、「時」のみを数値データとして抽出する
## hourに16が代入される
hour = date.hour
```



**2. timedeltaによる日時の加算、減算**

```
# 「年」「日」「時」「分」「秒」の各単位ごとに、日時の差分を指定できる

単位	引数名
年	years
日	days
時	hours
分	minutes
秒	seconds

# 例) 2時間分の差分を表すtimedeltaオブジェクトを生成する
two_hours = timedelta(hours=2)
```

**datetimeオブジェクトの日時情報に対して変更を加える**
```
# 「2016年1月1日16時0分0秒」のdatetimeオブジェクト
date = dt(2016, 1, 1, 16, 0, 0)

# 2時間後を表すdatetimeオブジェクトを生成
date_after2hours = date + timedelta(hours=2)

# 3年前を表すdatetimeオブジェクトを生成
date_before3years = date - timedelta(years=3)
```

In [ ]:
# ライブラリのインポート
from datetime import datetime as dt
from datetime import timedelta

# 「2016年1月1日16時0分0秒」の日時オブジェクトを作成する
date = dt(2016, 1, 1, 16, 0, 0)

# dateの１日後の日時オブジェクトdate_after1daysを作成する
date_after1days = date + timedelta(days=1)

### 1-7.衛星画像を並べて可視化する

**これまでの作業**
```
1.   f文字列の手法による文字列の置換
2.   datetimeモジュールによる日時の操作
```
**複数の画像を並べて表示する**

```
# 1枚目の画像を表示する
plt.subplot(1, 2, 1)
plt.imshow(1枚目の画像)

# 2枚目の画像を表示する
plt.subplot(1, 2, 2)
plt.imshow(2枚めの画像)

# plt.subplot(x, y, z)は、x行y列の図の内、z番目の図に描画することを宣言する
```




In [ ]:
# ライブラリのインポート
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
from datetime import datetime as dt
from datetime import timedelta


# 日時の初期設定として、1つ目の衛星画像に該当する日時を設定する
start_date = dt(2016, 1, 1, 16)

# 5時間分の衛星画像を順番に取得、描画する
for i in range(5):
    
    """
    2. 日時の操作
    """
    # start_dateから、i時間先の日時をdateに代入する
    date = start_date + timedelta(hours=i)
    
    # 年:year、月:month、日:day、時:hourの情報をdateから取得する
    year = date.year
    month = date.month
    day = date.day
    hour = date.hour
    
    # 2018年ならばテストデータ(test)
    # 2016年、2017年ならば学習データ(train)をphaseに代入する
    if year == 2018:
        phase = 'test'
    else:
        phase = 'train'
        
    """
    1. 文字列の置換
    """
    # ファイル名を指定する
    file_path = f"{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png"
    
    # 衛星画像を読み込む
    image = cv2.imread(file_path)
        
    # 1行5列並んだ図の内、i+1番目の図に描画する設定を行う 
    plt.subplot(1, 5, i+1)
    
    # 画像を描画する
    plt.imshow(image)

plt.show()

## 2.気象データの確認・把握

### 2-1.気象データのファイルパス

**各ディレクトリ名についての説明**
```
1.   train と test
学習用(train)として2016年&2017年、評価用(test)として2018年のデータを使用するため、2016年、2017年のデータはtrainディレクトリ内に、2018年のデータはtestディレクトリ内に含まれている。

2.   sat と met
衛星(Satellite)画像データを「sat」気象(Meteorological)データを「met」と、それぞれ表現している。
```

### 2-2.気象データを読み込む

**gzipファイルを扱う方針**
```
1.   ストレージ内に全てのgzipファイルを展開してしまう
# 読み込み処理にかかる時間が削減される一方、ストレージの容量が圧迫される

2.   気象データを読み込むタイミングで毎回gzipファイルを開く
# 読み込み処理にかかる時間がかかる一方、ストレージの容量が圧迫されない
```
**gzipファイルを引数に受け取り、NumPyファイルを返り値とする関数**
```
def Read_gz_Binary(file_path):
    file_tmp = file + "_tmp"
    with gzip.open(file, 'rb') as f_in:
        with open(file_tmp, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    bin_data = np.fromfile(file_tmp, np.float32)
    os.remove(file_tmp)
    return bin_data.reshape( [168,128] )
```

**Read_gz_Binary関数**

```
*   ファイルパスを引数に受け取り
*   形状が(168, 128)のNumPy配列を返り値とする
```
**Read_gz_Binary関数内部の処理**

**1.「gzip.open()」関数によるgzファイルの読み込み**
```
# gzipモジュールのインポート
import gzip

# gzipファイルをバイナリデータで開いたファイルオブジェクトとして返す
with gzip.open(ファイルパス, 'rb') as opened_file:

# 第1引数に、展開したいファイルのファイルパスを指定
# 第2引数は、'r'が読み込み、'b'がバイナリモードを意味する
```
**2.「shutil.copyfileobj()」関数によるファイルオブジェクトのコピー**

```
# shutilのインポート
import shutil

# ファイルオブジェクト1の中身を、ファイルオブジェクト2にコピーする
shutil.copyfileobj(ファイルオブジェクト1, ファイルオブジェクト2)
```
**「np.fromfile()」関数によるバイナリデータの読み込み**
```
# NumPyライブラリのインポート
import numpy as np

# バイナリファイルをNumPy配列として読み込む
bin_data = np.fromfile(ファイルパス, dtypeの指定)

# 第1引数にバイナリファイルのパス
# 第2引数に返り値となるNumPy配列のdtypeを指定する
```
**4. 「np.reshape()」関数による配列の2次元化**
```
# バイナリデータ(1次元の配列)を
# 2次元の配列に変換する
array2d = bin_data.reshape( [168,128] )
```
**Read_gz_Binary()関数の処理の流れ**
```
1.   「gzip.open()」関数によるgzファイルの読み込み
1.   「shutil.copyfileobj()」関数によるファイルオブジェクトのコピー
3.   「np.fromfile()」関数によるバイナリデータの読み込み
```
```
def Read_gz_Binary(file_path):

    # 1. 変数file_tmpに「元のファイル名_tmp」となる文字列を代入
    file_tmp = file_path + "_tmp"

    # 2. gzipファイルを、「読み込み(r)モード & バイナリ(b)モード」 で開き、
    #     開いたファイルオブジェクトをf_inに代入する
    with gzip.open(file_path, 'rb') as f_in:

        # 3. ファイル名を「元のファイル名_tmp」としたファイルを
        #     open()関数の「書き込み(w)モード & バイナリ(b)モード」で新規作成し、
        #     ファイルオブジェクトをf_outとする
        with open(file_tmp, 'wb') as f_out:

            # 4. f_inのファイルオブジェクトをf_outにコピーする
            shutil.copyfileobj(f_in, f_out)

    # 5. バイナリファイルをNumPy配列として読み込む
    met_data = np.fromfile(file_tmp, np.float32)

    # 6. 一時的に作成した展開済みのバイナリファイル「元のファイル名_tmp」を削除する
    os.remove(file_tmp)

    # 7. 読み込んだ配列の形状を(168, 128)に整形する
    met_data = met_data.reshape( [168,128] )

    return met_data
```



In [ ]:
# ライブラリのインポート
import numpy as np
import gzip
import shutil

# 気象データを読み込む関数; Read_gz_Binary を実装する
def Read_gz_Binary(file_path):
    file_tmp = file_path + "_tmp"
    with gzip.open(file_path, 'rb') as f_in:
        with open(file_tmp, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    # バイナリファイルをNumPy配列として読み込む
    met_data = np.fromfile(file_tmp, np.float32)
    os.remove(file_tmp)
    
    # 配列の次元に変更を加える
    met_data = met_data.reshape( [168,128] )
    
    return met_data

# 気象データのファイルパスを指定する
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

# 気象データを読み込む
met_data = Read_gz_Binary(file_path)

# 読み込んだ気象データに関する基本情報を出力する
print(type(met_data))
print(met_data.shape)

### 2-3.気象データを構成する値の統計量を確認する

**算出する統計量**

```
*   最大値
*   最小値
*   平均値
```

In [ ]:
# ライブラリのインポート
import numpy as np

# 気象データのファイルパスを指定する
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

# 気象データを読み込む
met_data = Read_gz_Binary(file_path)

# 最大値の出力
print('最大値: ', np.max(met_data))

# 最小値の出力
print('最小値: ', np.min(met_data))

# 平均値の出力
print('平均値: ', np.mean(met_data))

### 2-4.データの未計測部分を補間する

**1. 北側、南側の未計測部分を補間する(上下方向の補間)**
```
未計測部分
*   北側: 1行目と2行目
*   南側: 155行目〜168行目

有効な計測値
*   北側: 3行目
*   南側: 154行目


# 北側の未計測部分を補間する
data[0:2] = data[2]

# 南側の未計測部分を補間する
data[154:] = data[153]
```

**2. 西側の未計測部分を補間する(左右方向の補間)**
```
未計測部分
*   西側: 1列目〜8列目

有効な計測値
*   西側: 9列目


# 西側の未計測部分を補間する
data[:, :8] = data[:, 8].reshape(-1, 1)
```

In [ ]:
# ライブラリのインポート
import numpy as np

# 気象データのファイルパスを指定する
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

# 気象データを読み込む
met_data = Read_gz_Binary(file_path)

plt.subplot(1, 2, 1)
plt.imshow(met_data, 'gray')

# fill_lack_data()関数を実装する
def fill_lack_data(data):
    
    ## 1. 北側、南側の未計測部分を補間する(上下)
    
    # 北側の未計測部分を補間する
    data[0:2] = data[2]
    # 南側の未計測部分を補間する
    data[154:] = data[153]
    
    
    ## 2. 西側の未計測部分を補間する(左右)
    
    # 西側の未計測部分を補間する
    data[:, :8] = data[:, 8].reshape(-1, 1)
    
    return data

# fill_lack_data()関数をmet_dataに対して実行する
filled_data = fill_lack_data(met_data)



plt.subplot(1, 2, 2)
plt.imshow(filled_data, 'gray')

plt.show()

### 2-5.ファイルパスを一般化する

**構成成分の確認**
```
{train もしくは test}/met/{年}/{月}/{日}/{気象データの名称}.3.{年}{月}{日}{時}.gz

衛星画像データと同じく
*   train もしくは test
*   年
*   月
*   日
*   時
の5つに加えて、
*   気象データの名称
```
**気象データの種類名**
```
*   アルファベット名: 気象データの種類
*   数字: 等圧面における気圧


*   各アルファベット名と各種気象データの対応
アルファベット名	気象データの種類
HGT	高度
PRMSL	海面気圧
RH	湿度
TMP	気温
UGRD	東西風
VGRD	南北風
VVEL	鉛直流
```
**それぞれの変数の対応**
```
*   phase: train もしくは test
*   year: 年
*   month: 月
*   day: 日
*   hour: 時
*   data_type: 気象データの名称
```

In [ ]:
# ライブラリのインポート
from datetime import datetime as dt

# 日時オブジェクトの作成
date = dt(2016, 1, 1, 18)

"""
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

となるように、以下の変数phase, year, month, day, hour, data_typeに値を代入しましょう。
"""

phase = 'train'
year = date.year
month = date.month
day = date.day
hour = date.hour
data_type = 'HGT.200'



# f文字列を利用して、ファイルパスを表す文字列を作成する
file_path = f'{phase}/met/{year}/{month:02}/{day:02}/{data_type}.3.{year}{month:02}{day:02}{hour:02}.gz'


print(file_path)

# [2] データセットの作成

## 1.データが与えられている日付の取得

## 2.衛星画像データをまとめたnpyファイルの作成

## 3.気象データをまとめたnpyファイルの作成

# [3] 前処理

## 1.学習データと検証データの分割

## 2.データの正規化

## 3.入力データと正解データの分割

# [4] モデリング

## 1.Convolutional LSTMモデルの構築

## 2.コールバックの設定

## 3.モデルの学習と推論の実行

# [5] 予測精度の改善

## 1.特徴量として気象データを追加